Lets go through the steps that the robot takes during play.

Turn order:
(1) Determine the order of play - the robot doesn't need to know this until we start coding some AI decision making.

(2) Auction power plants
    Robot needs to know
     - what power plants are available
     - what power plants it has
     - what power plants other players have chosen for bidding
     - how many cities it has
    Robot will decide
     - if it needs to buy a new power plant or not
     - which power plant it will choose for bidding on
     - if it will join in the public bidding process
     - what the maximum it will pay for the power plant is
     - which power plant to discard if necessary
    Robot will update
     - what power plants it has
        - how many spaces it has for resources
     - how its resources are distributed
        - do any resources need throwing away
     - how much money it has

(3) Buying resources
    Robot needs to know
     - what space it has for resources
     - what resources it has
     - the cost of available resources
    Robot will decide
     - what resources it will buy
    Robot will update
     - what resources it has
     - where those resources are stored
     - how much money it has

(4) Building
    Robot needs to know
     - what the map is
        - what cities there are
        - what the connection costs are
     - what step we are in (step 1, 2 or 3)
     - what cities it has built on
     - what cities other players have built on
        - how many people have built there
        - what spaces are free, and their cost
     Robot will decide
      - if it will build a city
      - where to build the city
      - if it has enough money to build another city
     Robot will update
      - how much money it has
      - how many cities it has
        - where those cities are

(5) Bureaucracy
    Robot needs to know
     - how many cities it has built on
     - how many cities its power plants can supply
     - what resources it has
    Robot will decide
     - which resources to use to power the most cities
    The robot will then update
     - how much money it has
     - what resources it has
    


Variables and databases I need:
- a representation of the map
    - the cities on the map
    - the connections between the cities
    - the regions on the map
- the resource market
- the power plants
- a list of money earned depending on cities powered

- money
- each type of resource
- a list of power plants owned
- a list of cities in the robot's network

In [ ]:
# What libraries / functions do we need to import? Put them in here.

import mysql.connector
from mysql.connector import Error

from databaseConnection import create_db_connection, read_query



In [1]:
# Set the starting variables

robotMoney = 50
mapName = "Japan"
powerPlantDeck = "standard"
step = 1

# Database access details:
user_name = "Chris"
user_password = input("Please give the password for Chris")



# We will also need to set the robot options, but we'll code those later on

In [2]:
# Actually, we should initialise the instance of the Robot, so that it has money for the auction phase.

from robotClass import Robot

robot = Robot(robotMoney)

Robot.setRobotName(robot)

print(robot)


Harry has: Money = 50


In [3]:
# Phase 2: Power plant auction - create the power plant market

from powerPlants import PowerPlant, createMarket

# We ask the user to tell us the costs of the current power plants in the market:
listOfPowerPlants = createMarket(step)

power_plants = PowerPlant.updatePowerPlantMarket(user_name, user_password, listOfPowerPlants)

# We print some details to see what we have got.
print(listOfPowerPlants)
print(power_plants[1])


Market size is 8, because it is step 1.
MySQL Database connection successful
[1, 2, 3, 4, 5, 6, 8, 11]
Cost: 2, Type: Oil, Resource: oil, Resource2: None, PowerCost: 2, CitiesPowered: 1


In [4]:
# Now we will bid on some power plants
# We will start with the robot rule 'second smallest number'

# If I have a mix of 'print' and 'input' messages, the printed messages appear below this block, while the input
# messages appear in a box. This is confusing and hides some of the information I am trying to communicate.
# But if I switch it all to 'input' then no details are output below the block for me to read on later.
# hmmm......

# The robot will ask if it is their turn.
myTurn = input("Is it my turn to choose a power plant to bid on? Y or N?")

# If it is step 1 or 2, we need to split the existing market into the current and future market.
# If it is step 3, then all 6 power plants are in the current market
# Only the current market is of interest to the robot.

def createCurrentMarket(listOfPowerPlants):    
    if step == 3:
        currentMarket = listOfPowerPlants
    else:
        listOfPowerPlants.sort()
        currentMarket = listOfPowerPlants[:-4]
        
    return currentMarket

currentMarket = createCurrentMarket(listOfPowerPlants)
print(f"The current market is {currentMarket}")

currentMarketDetails = PowerPlant.updatePowerPlantMarket(user_name, user_password, currentMarket)

if myTurn.lower() == "y":
    choose = True
    while choose:
        choice = currentMarket[1]
        print(f"I bid {choice} electros for power plant {choice}.")
        otherBid = input(f"My bid is {choice} electros. Does anyone else want to bid on power plant {choice}? Y or N?")
        if otherBid.lower() == "y":
            print(f"Ok, I pass on {choice}.")
            listOfPowerPlants.remove(choice)
            newPowerPlant = int(input("What is the cost of the new power plant you have drawn?"))
            listOfPowerPlants.append(newPowerPlant)
            currentMarket = createCurrentMarket(listOfPowerPlants)
            print(f"The current market is {currentMarket}")
        elif otherBid.lower() == "n":
            choose = False
            print(f"Ok, I will buy power plant {choice} for {choice} electros")
            
# we need to create a function for the robot that buys the power plant
# that will reduce the money by the amount bid (which might not be the cost)
# and it will invoke a function to add the power plant to the list of power plants


The current market is [1, 2, 3, 4]
MySQL Database connection successful
I bid 2 electros for power plant 2.
Ok, I pass on 2.
The current market is [1, 3, 4, 5]
I bid 3 electros for power plant 3.
Ok, I pass on 3.
The current market is [1, 4, 5, 6]
I bid 4 electros for power plant 4.
Ok, I pass on 4.
The current market is [1, 5, 6, 8]
I bid 5 electros for power plant 5.
Ok, I will buy power plant 5 for 5 electros
